## MLflow Configuration

In [ ]:
#!pip install mlflow
#!pip install --upgrade jinja2
#!pip install --upgrade Flask
#!pip install setuptools

In [ ]:
## Import de librairie
import pandas as pd
import mlflow
from mlflow import MlflowClient
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

In [ ]:
## Connecxion au serveur MLFlow
client= MlflowClient(tracking_uri="http://127.0.0.1:8080")
## Configuration 
mlflow.set_tracking_uri("http://127.0.0.1:8080")


In [ ]:
# Récupération de toutes les expériences MLflow
all_experiments = client.search_experiments()
# Affichage des expériences 
print(all_experiments)

In [ ]:
# Recherche des expériences Mlflow Spécifique.
client.search_experiments(filter_string="name = 'RLG_Models'")

## Chargement du fichier 

In [ ]:
# Définition du chemin vers le fichier de  données 
data_dir = r"C:\Users\j_aka\Desktop\mlops\credit_risk_dataset.csv"
# Chargement de la données 
df = pd.read_csv(data_dir, delimiter=",") 
#Affichage des 3 premières lignes 
print(df.head(3).to_string())

## Regression logistic model

In [ ]:
# Définition des Features et de la cible 
target_col = 'loan_status'
X = df.drop(columns=['loan_status'])    
y = df['loan_status']

# Split des données 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Colonne numérique et colonne catégorielle 
num_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_features = X.select_dtypes(include=["object", "category"]).columns.tolist()

# Conversion des colonnes numériques en float64
X[num_features] = X[num_features].astype("float64")

#Encoder et Gérer les données Nan 
preprocessor = ColumnTransformer(
    transformers= [
        ("num_features", SimpleImputer(strategy="mean"), num_features),
        ("cat_features", OneHotEncoder(handle_unknown="ignore"),cat_features)
    ]
)


# Hyperparamètre du modèle
#params = {
   # "max_iter": 1000,
#}
# Nom du modèle : 
mlflow.set_experiment("RLG_Models")
# Nom du run pour cette itération d'entraînement
run_name = "RLG_Run"
# les artefacts du modèle
artifact_path = "RLG_artefacts"
# Fermer tout run précédent
#mlflow.end_run()

with mlflow.start_run(run_name=run_name):
    # Tag de version du modèle
    mlflow.set_tag("version", "v1.0")

# Encodage et entrainement du modèle 
    pipeline_fr = Pipeline(steps=[
    ("processor", preprocessor),
    ("model", LogisticRegression(max_iter=3000))
   ])
    
# Entrainement du modèle
    pipeline_fr.fit(X_train,y_train)

# Prédiction du modèle
    y_proba = pipeline_fr.predict_proba(X_val)[:, 1]
    y_pred = (y_proba >= 0.3).astype(int)

# Calcul des métriques
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    metrics = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1
    }

    mlflow.log_param("max_iter",1000)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(
        sk_model= pipeline_fr, 
        input_example=X_val, 
        name=artifact_path
    ) 

    print("✅ Modèle entraîné et logué dans MLflow avec succès.")
    print("\nRapport de classification :\n", classification_report(y_val, y_pred))
    print("\nMatrice de confusion :\n", confusion_matrix(y_val, y_pred))
    print("📊 Métriques :", metrics)